# Heat generation and storage subproblem validation

This model is the result of integrating the solar field, heat exchanger and thermal storage models. Depending on the version of the solar field model (direct, inverse), two subproblems are obtained.

In [97]:
from pathlib import Path
import time
import datetime
import numpy as np
import pandas as pd
from iapws import IAPWS97 as w_props
import hjson
from loguru import logger

# Visualization packages
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

from solarmed_modeling.utils import data_preprocessing, data_conditioning


# auto reload modules
%load_ext autoreload
%autoreload 2

# Paths definition
output_path: Path = Path("../../docs/models/attachments")
data_path: Path = Path("../../data")

date_str: str = "20231106" # '20230630'
filename_process_data = f'{date_str}_solarMED.csv'

# Available data to test
# data/calibration/20230807_aquasol.csv
# data/calibration/20230707_20230710_datos_tanques.csv
# Nextcloud/Juanmi_MED_PSA/EURECAT/data/20231030_solarMED.csv

sample_rate = '300s'
sample_rate_numeric = int(sample_rate[:-1])

# Resample figures using plotly_resampler
resample_figures: bool = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Pre-processing

In [60]:
with open( data_path / "variables_config.hjson") as f:
    vars_config = hjson.load(f)
    
with open(data_path/"plot_config.hjson") as f:
    plot_config = hjson.load(f)

# Load data and preprocess data
df = data_preprocessing(data_path / f"datasets/{filename_process_data}", vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(df, sample_rate_numeric=sample_rate_numeric, vars_config=vars_config)

2024-10-15 11:03:26.256 | INFO     | solarmed_modeling.utils:data_preprocessing:110 - Reading data from 20231106_solarMED.csv


2024-10-15 11:03:26.498 | DEBUG    | solarmed_modeling.utils:process_dataframe:66 - Index([], dtype='object')
2024-10-15 11:03:26.501 | INFO     | solarmed_modeling.utils:process_dataframe:73 - Number of duplicate index values in df: 0
2024-10-15 11:03:26.513 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:79 - Signal id HW1TT21 not found in dataframe columns.
2024-10-15 11:03:26.514 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:79 - Signal id FT-DES-002_VFD not found in dataframe columns.
2024-10-15 11:03:26.514 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:79 - Signal id vfd_prod_power not found in dataframe columns.
2024-10-15 11:03:26.515 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:79 - Signal id vfd_brine_power not found in dataframe columns.
2024-10-15 11:03:26.516 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:79 - Signal id vfd_sw_power not found in dataframe columns.
2024-10

## Test visualization

In [61]:
with open(data_path/"plot_config.hjson") as f:
    plot_config = hjson.load(f)
    
plots_to_remove: list[str] = ["costs", "three_way_valve", "med_flows", "med_temperatures", ]
[plot_config["plots"].pop(key, None) for key in plots_to_remove]

fig = experimental_results_plot(plot_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'thermal_storage_visualization_{df.index[0].strftime("%Y%m%d")}')
)

### Model definition

In [33]:
# Constants
# Not needed anymore, included in data preprocessing
# qsf_min = 6.2 * 6/100 # l/min -> m³/h, from 202409-25_27 test
# qsf_max = 148.8 * 6/100 # l/min -> m³/h, from 202409-25_27 test
# qts_src_min = 5 * 6/100 # l/min -> m³/h, from 20240925 test
# qts_src_max = 14 * 6/100 # l/min -> m³/h, from 20240925 test

In [31]:
from typing import Literal
from dataclasses import dataclass
from optimparallel import minimize_parallel
import scipy

from solarmed_modeling.metrics import calculate_metrics

from solarmed_modeling.solar_field import solar_field_model, ModelParameters as SfModParams
from solarmed_modeling.heat_exchanger import heat_exchanger_model, ModelParameters as HexModParams 
from solarmed_modeling.thermal_storage import thermal_storage_two_tanks_model, ModelParameters as TsModParams
from solarmed_modeling.thermal_storage.utils import Th_labels, Tc_labels

# logger.disable("solarmed_modeling.heat_exchanger")

# @dataclass
# class ModelParameters:
#     UA_hx: float
#     H_hx: float
#     H_sf: float
#     beta_sf: float
#     gamma_sf: float
#     UAts_h: float
#     UAts_c: float
#     Vts_h: float
#     Vts_c: float

@dataclass
class ModelParameters:
    sf: SfModParams
    ts: TsModParams
    hex: HexModParams

# class ComponentModel(NamedTuple):
#     model_fun: callable
#     model_params: ModelParameters

def heat_generation_and_storage_subproblem(
    qsf: np.ndarray[float], Tsf_in_ant: np.ndarray[float], Tsf_out_ant: float,
    qts_src: float, qts_dis: float,
    Tts_b_in: float, Tts_h: np.ndarray[float], Tts_c: np.ndarray[float], 
    Tamb: float, I: float,  
    model_params: ModelParameters,
    sample_time: int,
    water_props: tuple[w_props, w_props] = None,
    problem_type: Literal["1p2x", "2p1x"] = "1p2x",
    solver: Literal["scipy", "optimparallel"] = "optimparallel",
    solver_method: str = "lm",
) -> tuple[float, float, float, np.ndarray[float], np.ndarray[float]]:
    """
    Solves the heat generation and storage subproblem for a solar field and thermal storage system.
    Parameters:
    -----------
    qsf : float
        Solar field flow rate (m³/h).
    Tsf_in_ant : np.ndarray[float]
        Previous solar field inlet temperatures (ºC).
    Tsf_out_ant : float
        Previous solar field outlet temperature (ºC).
    qts_src : float
        Thermal storage charge flow rate (m³/h).
    qts_dis : float
        Thermal storage discharge flow rate (m³/h).
    Tts_b_in : float
        Bottom tank inlet temperature (ºC).
    Tts_h : np.ndarray[float]
        Previous hot tank temperatures (ºC).
    Tts_c : np.ndarray[float]
        Previous cold tank temperatures (ºC).
    Tamb : float
        Ambient temperature (ºC).
    I : float
        Solar irradiance (W/m²).
    model_params : ModelParameters
        Model parameters for the system.
    # component_models : list[ComponentModel]
    #     list containing one element with the model function and a parameters 
    #     dataclass for each of the subsystem components, in the following order:
    #     0: heat generation, 1: heat exchange, 3: heat storage
    sample_time : int
        Sample time (seconds).
    water_props : tuple[w_props, w_props], optional
        Water properties for the hot and cold tanks.
    problem_type : Literal["1p2x", "2p1x"], optional
        Type of problem to solve. Default is "1p2x" (1 problem, 2 variables).
    solver : Literal["scipy", "optimparallel"], optional
        Solver to use for optimization. Default is "optimparallel".
    Returns:
    --------
    tuple[float, float, float, np.ndarray[float], np.ndarray[float]]
        Tuple containing estimations for the solar field inlet temperature, solar field outlet temperature,
        thermal storage tank inlet temperature, hot tank temperatures, and cold tank temperatures.
    """    

    def inner_function(x, return_states: bool = False):
        """
        Variables that end with an underscore are the ones calculated in the 
        inner function, to avoid overwriting the outer scope variables.
        """
        
        if len(x) == 2:
            Tsf_out = x[0]
            Tts_c_b = x[1]
        elif len(x) == 1:
            # Bottom tank temperature is not considered to change
            Tsf_out = x[0]
            Tts_c_b = Tts_c_b_orig
        else:
            raise ValueError("Invalid number of decision variables")

        # Heat exchanger of solar field - thermal storage
        Tsf_in_, Tts_t_in_ = heat_exchanger_model(
            Tp_in=Tsf_out,  # Solar field outlet temperature (decision variable, ºC)
            Ts_in=Tts_c_b,  # Cold tank bottom temperature (ºC)
            qp=qsf[-1],  # Solar field flow rate (m³/h)
            qs=qts_src,  # Thermal storage charge flow rate (decision variable, m³/h)
            Tamb=Tamb,
            
            UA=model_params.hex.UA,
            H=model_params.hex.H,
            water_props=water_props,
        )

        # Solar field
        Tsf_in_ = np.append(Tsf_in_ant, Tsf_in_)

        Tsf_out_ = solar_field_model(
            Tin=Tsf_in_,
            q=qsf,
            I=I,
            Tamb=Tamb,
            Tout_ant=Tsf_out_ant,
            
            beta=model_params.sf.beta,
            H=model_params.sf.H,
            gamma=model_params.sf.gamma,
            water_props = water_props[0],
            sample_time=sample_time,
            consider_transport_delay=True,
        )

        # Thermal storage
        Tts_h_, Tts_c_ = thermal_storage_two_tanks_model(
            Ti_ant_h=Tts_h, Ti_ant_c=Tts_c,  # [ºC], [ºC]
            Tt_in=Tts_t_in_,  # ºC
            Tb_in=Tts_b_in,  # ºC
            Tamb=Tamb,  # ºC
            qsrc=qts_src,  # m³/h
            qdis=qts_dis,  # m³/h
            
            UA_h=model_params.ts.UA_h,  # W/K
            UA_c=model_params.ts.UA_c,  # W/K
            Vi_h=model_params.ts.V_h,  # m³
            Vi_c=model_params.ts.V_c,  # m³
            water_props=water_props,
            ts=sample_time, # seg 
            Tmin=Tmin  # ºC
        )
        Tts_c_b_ = Tts_c_[-1]

        if return_states:
            return Tsf_in_[-1], Tsf_out_, Tts_t_in_, Tts_h_, Tts_c_
        elif len(x) == 2:
            return np.array( [abs(Tsf_out - Tsf_out_), abs(Tts_c_b - Tts_c_b_) ])
        elif len(x) == 1:
            return [abs(Tsf_out - Tsf_out_)]
        else:
            raise ValueError("Invalid number of decision variables")
    # End of inner function ---------------------------------------------------
           

    if problem_type != "1p2x":
        raise NotImplementedError("Currently, only `1p2x` alternative is implemented")
    
    if water_props is None:
        # Initialize from input values
        water_props = (
            w_props(P=0.2, T=Tts_h[0] + 273.15),
            w_props(P=0.2, T=Tts_c[-1] + 273.15)
        )
    
    Tts_c_b_orig: float | None = None
    if problem_type == "1p2x":
        pass
    elif problem_type == "2p1x":
        Tts_c_b_orig = float(Tts_c[-1]) # To have an inmutable value
    else:
        raise ValueError("Invalid problem type")
    
    # Cap solar field outlet temperature
    if Tsf_out_ant > Tmax:
        Tsf_out_ant = Tmax
        
    initial_guess = [Tsf_out_ant, Tts_c[-1]]
    bounds = ((Tmin, Tmin), (Tmax, Tmax)) if solver_method != "lm" else None 

    if solver == "scipy":
        outputs = scipy.optimize.least_squares(fun=inner_function, x0=initial_guess, bounds=bounds, xtol=1e-1, ftol=1e-1, method=solver_method)
    elif solver == "optimparallel":
        outputs = minimize_parallel(fun=inner_function, x0=initial_guess, bounds=bounds, tol=1e-1)
    else:
        raise ValueError(f"Invalid solver {solver}, options are: 'scipy', 'optimparallel'")
    
    # Cap solar field outlet temperature
    if outputs.x[0] > Tmax:
        outputs.x[0] = Tmax
    
    # With the system of equations solved, calculate the outputs
    return inner_function(outputs.x, return_states=True)
        
# Uncomment and comment the rest to test particular steps
# Tsf_in, Tsf_out, Tts_t_in, Tts_h, Tts_c = heat_generation_and_storage_subproblem(
#         # Solar field
#         qsf= df.iloc[i-span: i]["qsf"].values,
#         Tsf_in_ant = Tsf_in_ant,
#         Tsf_out_ant= Tsf_out_ant,
        
#         # Thermal storage
#         qts_src= ds["qts_src"], qts_dis= ds["qts_dis"],
#         Tts_b_in= ds["Tts_c_in"], 
#         Tts_h= Tts_h, 
#         Tts_c= Tts_c, 
        
#         # Environment
#         Tamb=ds["Tamb"], I=ds["I"],  
        
#         # Parameters
#         model_params= model_params, sample_time = sample_rate_numeric,
#         water_props = water_props,
#         problem_type = "1p2x",
#         solver = "scipy",
#     )

Tmax = 110 # Make sure to estimate water properties with a high enough pressure to avoid phase changes
Tmin = 0



IndexError: index -1 is out of bounds for axis 0 with size 0

### Model evaluation

In [34]:
out_mod.shape
print(f"{span=}, {idx_start=}")
print(df["Tsf_in"].values.shape)
print(len(df) - idx_start)
print(0,idx_start-1)

span=1, idx_start=np.int64(2)
(67,)
65
0 1


In [38]:
# Uncomment to test particular steps
# Tsf_in, Tsf_out, Tts_t_in, Tts_h, Tts_c = heat_generation_and_storage_subproblem(
#     # Solar field
#     qsf= df.iloc[i-span: i]["qsf"].values,
#     Tsf_in_ant = Tsf_in_ant,
#     Tsf_out_ant= Tsf_out_ant,
    
#     # Thermal storage
#     qts_src= ds["qts_src"], qts_dis= ds["qts_dis"],
#     Tts_b_in= ds["Tts_c_in"], 
#     Tts_h= Tts_h, 
#     Tts_c= Tts_c, 
    
#     # Environment
#     Tamb=ds["Tamb"], I=ds["I"],  
    
#     # Parameters
#     model_params=model_params,
#     # model_params= model_params, 
#     sample_time = sample_rate_numeric,
#     water_props = water_props,
#     problem_type = "1p2x",
#     solver = "scipy",
#     solver_method = "dogbox"
# )

In [95]:
from solarmed_modeling.heat_gen_and_storage import (ModelParameters, 
                                                    heat_generation_and_storage_subproblem)
from solarmed_modeling.heat_gen_and_storage.utils import out_var_ids
from solarmed_modeling.thermal_storage.utils import Th_labels, Tc_labels
import math

model_params = ModelParameters()

span = math.ceil(600 / sample_rate_numeric) # 600 s
idx_start = np.max([span, 2]) # idx_start-1 should at least be one 

# if span >= idx_start:
#     logger.warning(
#         f"Span {span} can't be greater or equal than idx_start {idx_start}. Increasing idx_start by 1"
#     )
#     span = idx_start
#     idx_start = idx_start+1

out_mod = np.zeros((len(df) - idx_start, len(out_var_ids)), dtype=float)

# Signal filtering
# df.loc[df["qsf"] < qsf_min, "qsf"] = 0
# df["qsf"] = df["qsf"].clip(upper=qsf_max)
# df.loc[df["qts_src"] < qts_src_min, "qts_src"] = 0
# df["qts_src"] = df["qts_src"].clip(upper=qts_src_max)

# Initial values
Tsf_in_ant = df.iloc[0:idx_start-1]["Tsf_in"].values
out_mod[0,0] = df.iloc[idx_start]["Tsf_in"]
out_mod[0,1] = df.iloc[idx_start]["Tsf_out"]
out_mod[0,3] = df.iloc[idx_start]["Tts_h_in"]
idx_span: list[int, int] = [None, None]
idx_span[0] = 3
idx_span[1] = idx_span[0] + len(Th_labels)
out_mod[0, idx_span[0]:idx_span[1]] = np.array([df[label].values[idx_start] for label in Th_labels])
idx_span[0] = idx_span[1]
idx_span[1] = idx_span[0] + len(Tc_labels)
out_mod[0, idx_span[0]:idx_span[1]] = np.array([df[label].values[idx_start] for label in Tc_labels])

water_props: tuple[w_props, w_props] = (
    w_props(P=0.2, T=90 + 273.15), # P=2 bar  -> 0.2MPa, T in K, average working temperature of hot tank
    w_props(P=0.2, T=65 + 273.15)  # P=2 bar  -> 0.2MPa, T in K, average working temperature of cold tank
)

start_time_eval = time.time()
for i in range(idx_start + 1, len(df)):
    ds = df.iloc[i]
    j = i - idx_start
    start_time = time.time()
    
    Tsf_in_ant = np.roll(Tsf_in_ant, -1)
    Tsf_in_ant[-1] = out_mod[j-1, 0]
    Tsf_out_ant= out_mod[j-1, 1]
    idx_span[0] = 3
    idx_span[1] = idx_span[0] + len(Th_labels)
    Tts_h= out_mod[j-1, idx_span[0]:idx_span[1]]
    idx_span[0] = idx_span[1]
    idx_span[1] = idx_span[0] + len(Tc_labels)
    Tts_c= out_mod[j-1, idx_span[0]:idx_span[1]]
    
    Tsf_in, Tsf_out, Tts_t_in, Tts_h, Tts_c = heat_generation_and_storage_subproblem(
        # Solar field
        qsf= df.iloc[i-span: i]["qsf"].values,
        Tsf_in_ant = Tsf_in_ant,
        Tsf_out_ant= Tsf_out_ant,
        
        # Thermal storage
        qts_src= ds["qts_src"], qts_dis= ds["qts_dis"],
        Tts_b_in= ds["Tts_c_in"], 
        Tts_h= Tts_h, 
        Tts_c= Tts_c, 
        
        # Environment
        Tamb=ds["Tamb"], I=ds["I"],  
        
        # Parameters
        model_params=model_params,
        # model_params= model_params, 
        sample_time = sample_rate_numeric,
        water_props = water_props,
        problem_type = "1p2x",
        solver = "scipy",
        solver_method = "dogbox"
    )
    
    out_mod[j, 0] = Tsf_in
    out_mod[j, 1] = Tsf_out
    out_mod[j, 2] = Tts_t_in
    idx_span[0] = 3
    idx_span[1] = idx_span[0] + len(Th_labels)
    out_mod[j, idx_span[0]:idx_span[1]] = Tts_h
    idx_span[0] = idx_span[1]
    idx_span[1] = idx_span[0] + len(Tc_labels)
    out_mod[j, idx_span[0]:idx_span[1]] = Tts_c

df_mod = pd.DataFrame(
    out_mod, columns=out_var_ids, index=df.index[idx_start:]
).assign(
    Thx_p_in =lambda df: df["Tsf_out"],
    Thx_p_out=lambda df: df["Tsf_in"],
    Thx_s_in =lambda df: df["Tts_c_b"],
    Thx_s_out=lambda df: df["Tts_h_in"]
)

# When secondary flow is null, equating HEX temperatures to solar field is more accurate
df_mod.loc[df["qts_src"] < 0.1, "Thx_s_in"] = df_mod.loc[df["qts_src"] < 0.1, "Thx_p_in"]

metrics = calculate_metrics(df.iloc[idx_start:][out_var_ids].values, out_mod)
logger.info(f"Completed model evaluation. Elapsed time: {time.time() - start_time_eval:.2f} s, MAE: {metrics['MAE']:.2f} ºC")

2024-10-15 12:22:48.318 | INFO     | __main__:<module>:109 - Completed model evaluation. Elapsed time: 1.00 s, MAE: 5.39 ºC


In [96]:
with open(data_path/"plot_config_validation.hjson") as f:
    plot_config_val = hjson.load(f)
    
keys_to_del: list[str] = ["costs", "heat_exchanger_power", "thermal_storage_power_balance", "three_way_valve", "med_flows", "med_temperatures"]
[plot_config_val["plots"].pop(key, None) for key in keys_to_del]

fig = experimental_results_plot(
    plot_config_val,
    df,
    df_comp=df_mod,
    vars_config=vars_config,
    resample=resample_figures,
)

fig.show(
    config=generate_plotly_config(
        fig, figure_name=f'heat_gen_stge_{df.index[0].strftime("%Y%m%d")}'
    )
)

### Evaluate model